In [1]:
import numpy as np
import pandas as pd
import streamlit as st
from datetime import datetime, timedelta
import os
import gspread
from google.oauth2 import service_account
from gspread_dataframe import set_with_dataframe
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from configs import *
import requests

In [2]:
def connect_to_gs(_service_account_key):
    scopes = ["https://www.googleapis.com/auth/spreadsheets"]
    credentials = service_account.Credentials.from_service_account_info(_service_account_key, scopes=scopes)
    gs_connection = gspread.authorize(credentials)
    return gs_connection

gs_connection = connect_to_gs(st.secrets["gcp_service_account"])

In [3]:
def fetch_max_gw(_gc, sheet_name, sheet_key):
    try:
        # Open specific sheet
        gs = _gc.open_by_key(sheet_key)

        # Open specific tab within the sheet
        tab = gs.worksheet(sheet_name)

        data = tab.get_all_values()
        headers = data.pop(0)
        df = pd.DataFrame(data, columns=headers)

        # Convert the first column to numeric
        df[headers[0]] = pd.to_numeric(df[headers[0]])

        # Get the maximum value of the first column
        max_value = df[headers[0]].max()

        return int(max_value)

    except gspread.exceptions.APIError as e:
        print("Error accessing Google Sheets API:", e)
        return None
    except gspread.exceptions.WorksheetNotFound as e:
        print("Error: Worksheet not found:", e)
        return None
    except Exception as e:
        print("An error occurred:", e)
        return None

In [4]:
def update(_gc):
    with requests.Session() as session:
        general_response = session.get(general_endpoint).json()

    events = pd.DataFrame(general_response['events'])[['id', 'is_previous', 'is_current', 'is_next', 'finished', 'data_checked',]]
    if len(events[events['is_current'] == True]) == 0:
        print('season hasn\'t started pull placeholder data')
        gw = 
        return False, gw
    
    gw = int(events[events["is_current"] == True]["id"])
    finished = bool(events[events["is_current"] == True]["finished"].values)
    data_checked = bool(events[events["is_current"] == True]["data_checked"].values)
    
    if finished and data_checked:
        print(f"finished: {finished}\nchecked: {data_checked}\ngame week: {gw}")
        max_stored_gw = fetch_max_gw(_gc, gameweek_results_table, prod_google_sheet_key)
        if max_stored_gw < gw:      
            print('max stored_gw < current week')
            print('we will update here')
            return True, gw
        
        else: 
            print('data is already latest - pull from gs')
            return False, max_stored_gw
    else:
        print("its either a new week or the old week hasnt completetly finished - pull from gs")
        print(f"finished: {finished}\nchecked: {data_checked}\ngame week: {gw}")
        return False, gw

In [5]:
x , current_gw = update(gs_connection)

season hasn't started pull placeholder data


In [6]:
current_gw

1

In [7]:
def fetch_gameweek_data(_gc, sheet_name, sheet_key, columns_list):
    try:
        # Open specific sheet
        gs = _gc.open_by_key(sheet_key)

        # Open specific tab within the sheet
        tab = gs.worksheet(sheet_name)

        data = tab.get_all_values()
        headers = data.pop(0)
        df = pd.DataFrame(data, columns=headers)

        # to handle numeric columns that are imported as strings
        for column in columns_list:
            df[column] = pd.to_numeric(df[column])

        return df

    except gspread.exceptions.APIError as e:
        print("Error accessing Google Sheets API:", e)
        return None
    except gspread.exceptions.WorksheetNotFound as e:
        print("Error: Worksheet not found:", e)
        return None
    except Exception as e:
        print("An error occurred:", e)
        return None


In [8]:
gameweek_df = fetch_gameweek_data(
        gs_connection,
        gameweek_results_table,
        prod_google_sheet_key,
        ["event", "points", "total_points", "event_transfers_cost", "points_on_bench"],)

In [11]:
df = gameweek_df

In [12]:
current_gw_df = df[df['event'] == current_gw]

In [13]:
current_gw_df

,event,entry,player_name,points,total_points,event_transfers_cost,points_on_bench


In [9]:
def get_first_last(df, current_gw):
    # Filter the DataFrame for the given current_gw
    current_gw_df = df[df['event'] == current_gw]

    # Sort the filtered DataFrame based on points, total_points, and player_name
    sorted_df = current_gw_df.sort_values(by=["points", "total_points", "player_name"],
                                           ascending=[False, False, True])

    # Get the player names of the first and last place
    first_place_name = sorted_df['player_name'].iloc[0]
    last_place_name = sorted_df['player_name'].iloc[-1]

    return first_place_name, last_place_name

In [10]:
first_place, last_place = get_first_last(gameweek_df, current_gw)

IndexError: single positional indexer is out-of-bounds